In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo, lo, tools
import scipy


def get_hamiltonian(mol,orth):
    '''Get the hamiltonian matrix elements with respect to the orthogonalized basis'''
    hcore = mol.intor_symmetric('int1e_kin') + mol.intor_symmetric('int1e_nuc')
    hcore_orth = orth.conj().T @ hcore @ orth
    #hcore_orth = np.einsum('pi,pq,qj->ij', orth, hcore, orth)
    eri = mol.ao2mo(orth)
    #Not imposing symmetry here
    eri = ao2mo.restore(1, np.asarray(eri), mol.nao)
    return hcore_orth, eri

def read_fcidump(fname, norb):
    """

    :param fname: electron integrals dumped by pyscf
    :param norb: number of orbitals
    :return: electron integrals for 2nd quantization with chemist's notation
    """
    v2e = np.zeros((norb, norb, norb, norb))
    h1e = np.zeros((norb, norb))

    with open(fname, "r") as f:
        lines = f.readlines()
        for line, info in enumerate(lines):
            if line < 4:
                continue
            line_content = info.split()
            integral = float(line_content[0])
            p, q, r, s = [int(i_index) for i_index in line_content[1:5]]
            if r != 0:
                # v2e[p,q,r,s] is with chemist notation (pq|rs)=(qp|rs)=(pq|sr)=(qp|sr)
                v2e[p-1, q-1, r-1, s-1] = integral
                v2e[q-1, p-1, r-1, s-1] = integral
                v2e[p-1, q-1, s-1, r-1] = integral
                v2e[q-1, p-1, s-1, r-1] = integral
            elif p != 0:
                h1e[p-1, q-1] = integral
                h1e[q-1, p-1] = integral
            else:
                nuc = integral
    return h1e, v2e, nuc

def hamiltonian_integral(mol):
    # 1e & 2e integrals
    s_mat = mol.intor('int1e_ovlp')
    ao_coeff = lo.orth.lowdin(s_mat)
    norb = ao_coeff.shape[0]
    import tempfile
    ftmp = tempfile.NamedTemporaryFile()
    tools.fcidump.from_mo(mol, ftmp.name, ao_coeff)
    h1e, eri, nuc = read_fcidump(ftmp.name, norb)
    # Cholesky decomposition
    v2e = eri.reshape((norb**2, -1))
    u, s, v = scipy.linalg.svd(v2e)
    l_tensor = u * np.sqrt(s)
    l_tensor = l_tensor.T
    l_tensor = l_tensor.reshape(l_tensor.shape[0], norb, norb)
    return h1e, eri, nuc, l_tensor

mol = gto.Mole()
mol.build(
    atom = 'H 0 0 0; F 0 0 1.1',  # in Angstrom
    basis = 'ccpvdz',
    symmetry = True,
)

ovlp_mat = mol.intor('int1e_ovlp')
X = lo.orth.lowdin(ovlp_mat)

hcore, myeri = get_hamiltonian(mol,X)
h1e, eri, nuc, l_tensor = hamiltonian_integral(mol)

print(hcore.shape)
print(np.linalg.norm(hcore-h1e))
print(np.linalg.norm(myeri-eri))
print(nuc - mol.energy_nuc())

(19, 19)
8.816090421010383e-15
1.3076028294047347e-14
0.0


/Users/formic/pyscf/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '
